<!-- Import packages -->

## Install and import

Install packages

In [1]:
pip install -r Requirements.txt

ERROR: Could not find a version that satisfies the requirement mdfile (from versions: none)
ERROR: No matching distribution found for mdfile
Note: you may need to restart the kernel to use updated packages.


Import libraries


In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import json
import datetime as dt
import requests
import os
import glob

## S&P500 data

Create a date range

In [3]:
end_date = dt.datetime.now()
start_date = dt.datetime(1900, 1, 1)

Pull S&P500 data using API from rapidapi.com

In [4]:
url = "https://s-p-500-history-api.p.rapidapi.com/historical-stock-data.php"

querystring = {"interval":"day","symbol":"^GSPC","start":start_date,"end":end_date}

headers = {
	"x-rapidapi-key": "1818711435msh7ff1e71cfe61ec4p1b64e1jsn32c289b7a487",
	"x-rapidapi-host": "s-p-500-history-api.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)


Extract data from json file

In [5]:
response_json = json.loads(response.text)
sp = pd.DataFrame(response_json['data'])
sp.head()

,date_time,timestamp,high,low,close,open,volume,adjusted_close
0,"December 30, 1927, 2:30 PM UTC",-1325583000,17.660000,17.660000,17.660000,17.660000,0,17.660000
1,"January 3, 1928, 2:30 PM UTC",-1325237400,17.760000,17.760000,17.760000,17.760000,0,17.760000
2,"January 4, 1928, 2:30 PM UTC",-1325151000,17.719999,17.719999,17.719999,17.719999,0,17.719999
3,"January 5, 1928, 2:30 PM UTC",-1325064600,17.549999,17.549999,17.549999,17.549999,0,17.549999
4,"January 6, 1928, 2:30 PM UTC",-1324978200,17.660000,17.660000,17.660000,17.660000,0,17.660000


Some of the data is missing the data from the 'volume' column. I will drop these rows.

In [6]:
sp = sp[sp['volume']!=0]
sp = sp.reset_index(drop=True)
sp

,date_time,timestamp,high,low,close,open,volume,adjusted_close
0,"January 3, 1950, 2:30 PM UTC",-630927000,16.660000,16.660000,16.660000,16.660000,1260000,16.660000
1,"January 4, 1950, 2:30 PM UTC",-630840600,16.850000,16.850000,16.850000,16.850000,1890000,16.850000
2,"January 5, 1950, 2:30 PM UTC",-630754200,16.930000,16.930000,16.930000,16.930000,2550000,16.930000
3,"January 6, 1950, 2:30 PM UTC",-630667800,16.980000,16.980000,16.980000,16.980000,2010000,16.980000
4,"January 9, 1950, 2:30 PM UTC",-630408600,17.080000,17.080000,17.080000,17.080000,2520000,17.080000
...,...,...,...,...,...,...,...,...
18928,"March 27, 2025, 1:30 PM UTC",1743082200,5732.279785,5670.939941,5693.310059,5695.640137,4403460000,5693.310059
18929,"March 28, 2025, 1:30 PM UTC",1743168600,5685.890137,5572.419922,5580.939941,5679.200195,4513330000,5580.939941
18930,"March 31, 2025, 1:30 PM UTC",1743427800,5627.560059,5488.729980,5611.850098,5527.910156,5257250000,5611.850098
18931,"April 1, 2025, 1:30 PM UTC",1743514200,5650.569824,5558.520020,5633.069824,5597.529785,4434500000,5633.069824


Drop 'timestamp' and 'adjusted close' columns

In [7]:
sp.drop(columns=['timestamp','adjusted_close'], inplace=True, axis=1)

In [8]:
# Below is attempt at using FRED website API to pull data. I'm going to use batch downloads instead, but keeping this code in case that changes.

In [9]:
# URL_BASE = 'https://api.stlouisfed.org/'
# ENDPOINT = 'fred/series/observations'

# URL = URL_BASE + ENDPOINT

# API_KEY = '9de274aca217f6c8787ddb88fbcf323c'
# INDICATORS = 'T10Y2Y'

# params = {
#     'api_key': API_KEY,
#     'series_id': INDICATOR,
#     'file_type': 'json',
#     'observation_start': start_date,
#     'observation_end': end_date
# }

# requests = requests.get(URL, params=params)
# data = res.json()
# data = data['observations']

In [10]:
# URL_BASE = 'https://api.stlouisfed.org/'
# ENDPOINT = 'fred/series/observations'

# URL = URL_BASE + ENDPOINT

# API_KEY = '9de274aca217f6c8787ddb88fbcf323c'
# INDICATORS = ['T10Y2Y','FEDFUNDS']

# params = {
#     'api_key': API_KEY,
#     'series_id': INDICATOR,
#     'file_type': 'json',
#     'observation_start': start_date,
#     'observation_end': end_date,
#     'frequency': 'd'
# }

# requests = requests.get(URL, params=params)
# data = res.json()
# data = data['observations']

## Collect macroeconomic FRED data

Data details get using BeautifulSoup


In [11]:
from bs4 import BeautifulSoup

url_list = [
"https://fred.stlouisfed.org/series/PAYEMS",
"https://fred.stlouisfed.org/series/UNRATE",
"https://fred.stlouisfed.org/series/GDP",
"https://fred.stlouisfed.org/series/A191RP1Q027SBEA",
"https://fred.stlouisfed.org/series/M2SL",
"https://fred.stlouisfed.org/series/CPIAUCSL",
"https://fred.stlouisfed.org/series/HOUST",
"https://fred.stlouisfed.org/series/SAHMCURRENT",
"https://fred.stlouisfed.org/series/LORSGPORUSQ659S",
"https://fred.stlouisfed.org/series/BOGZ1FL135010005Q",
"https://fred.stlouisfed.org/series/UMCSENT",
"https://fred.stlouisfed.org/series/CE16OV",
"https://fred.stlouisfed.org/series/W875RX1#",
"https://fred.stlouisfed.org/series/INDPRO"
]

url_print = []
title_print = []
notes_text_print = []

# Loop through the URLs and extract the title and paragraph under "Notes"
for url in url_list:
# Extract the title 
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    title = soup.title.string.strip()
# Extract the paragraph under "Notes"
    notes = soup.find("div", id="notes") 
    notes_text = notes.get_text(strip=True) 

    # Create the lists and print the results
    url_print.append(f"URL: {url}")
    title_print.append(f"Title: {title}")
    notes_text_print.append(f"Notes: {notes_text}\n")

    print(f"URL: {url}")
    print(f"Title: {title}")  
    print(f"Notes: {notes_text}\n")

URL: https://fred.stlouisfed.org/series/PAYEMS
Title: All Employees, Total Nonfarm (PAYEMS) | FRED | St. Louis Fed
Notes: NotesSource:U.S. Bureau of Labor StatisticsRelease:Employment SituationUnits:Thousands of Persons, Seasonally AdjustedFrequency:MonthlyNotes:All Employees: Total Nonfarm, commonly known as Total Nonfarm Payroll, is a measure of the number of U.S. workers in the economy that excludes proprietors, private household employees, unpaid volunteers, farm employees, and the unincorporated self-employed. This measure accounts for approximately 80 percent of the workers who contribute to Gross Domestic Product (GDP).This measure provides useful insights into the current economic situation because it can represent the number of jobs added or lost in an economy. Increases in employment might indicate that businesses are hiring which might also suggest that businesses are growing. Additionally, those who are newly employed have increased their personal incomes, which means (all 

Read CSV files into variables

In [ ]:
# Loop through the csv files in the 'data' folder and read_csv them into dictionary
dataframes_dict = {}
folder = 'data/FRED Data'
for file in os.listdir(folder):
    file_name = os.path.splitext(file)[0]
    file_path = os.path.join(folder, file)
    if file.endswith('.csv'):
        dataframes_dict[file_name] = pd.read_csv(file_path)


In [15]:
for names, data in dataframes_dict.items():
    locals()[names] = data
    print(names)

for names, data in dataframes_dict.items():
    locals()[names] = data
    print(data.info())

M2SL
UNRATE
A191RP1Q027SBEA
GDP
LORSGPORUSQ659S
Presidential_Results
PAYEMS
CPIAUCSL
UMCSENT
INDPRO
BOGZ1FL135010005Q
W875RX1
CE16OV
DFF
SAHMCURRENT
CMDEBT
HOUST
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 794 entries, 0 to 793
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   observation_date  794 non-null    object 
 1   M2SL              794 non-null    float64
dtypes: float64(1), object(1)
memory usage: 12.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 926 entries, 0 to 925
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   observation_date  926 non-null    object 
 1   UNRATE            926 non-null    float64
dtypes: float64(1), object(1)
memory usage: 14.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 2 columns):
 #   Column            Non-Null Co

## Clean FRED Data

I will go through each database (created above) and clean the data

,observation_date,HOUST
0,1959-01-01,1657
1,1959-02-01,1667
2,1959-03-01,1620
3,1959-04-01,1590
4,1959-05-01,1498
...,...,...
789,2024-10-01,1344
790,2024-11-01,1305
791,2024-12-01,1526
792,2025-01-01,1350
